In [1]:
import xarray as xr
import numpy as np
import scipy.stats as st

In [2]:
mods = ['bcc-csm1-1','CanESM2','CCSM4','CNRM-CM5','CSIRO-Mk3-6-0','GFDL-CM3','GISS-E2-R','HadGEM2-ES','inmcm4','IPSL-CM5A-LR','MIROC-ESM','MIROC5','MRI-CGCM3','NorESM1-M']

In [3]:
#Get 1x1 degree grid
with xr.open_dataset('../rawdata/rsl_85_2090.nc') as ds:
    lon = ds['x'].values
    lat = ds['y'].values
    slr_md = ds['slr_md'].values

In [4]:
def get_vars(scen):
    stdn = np.zeros((len(mods),len(lat),len(lon)))    
    glac = np.zeros((len(mods),len(lat),len(lon)))
    gsmb = np.zeros((len(mods),len(lat),len(lon)))    
    asmb = np.zeros((len(mods),len(lat),len(lon)))
    adyn = np.zeros((len(mods),len(lat),len(lon)))    
    
    for m,model in enumerate(mods):
        #Read RSL
        with xr.open_dataset(f'../data/rsl/{model}_{scen}.nc') as ds:
            stdn[m,:,:] = np.nanmean(ds['stdn'][-20:,:,:].values,axis=0)
            glac[m,:,:] = np.nanmean(ds['glac'][-20:,:,:].values,axis=0)            
            gsmb[m,:,:] = np.nanmean(ds['gsmb'][-20:,:,:].values,axis=0)            
            asmb[m,:,:] = np.nanmean(ds['asmb'][-20:,:,:].values,axis=0)            
            adyn[m,:,:] = np.nanmean(ds['adyn'][-20:,:,:].values,axis=0)
            
    return stdn,glac,gsmb,asmb,adyn

In [5]:
def calc_corr(var1,var2):
    corr = np.zeros((len(lat),len(lon)))
    cov  = np.zeros((len(lat),len(lon)))
    
    for j in range(len(lat)):
        for i in range(len(lon)):
            corr[j,i] = st.spearmanr(var1[:,j,i],var2[:,j,i])[0]
            sig1 = np.nanstd(var1[:,j,i])
            sig2 = np.nanstd(var2[:,j,i])
            cov[j,i] = 2*corr[j,i]*sig1*sig2

    vslr = np.nanvar(stdn+glac+gsmb+asmb+adyn,axis=0)
    rcov = 100.*cov/vslr #Relative to ensemble variance in percent
            
    corr[np.isnan(slr_md)] = np.nan
    rcov[np.isnan(slr_md)] = np.nan
    print('got one')
    return corr,rcov

In [6]:
for scen in ['rcp45','rcp85']:
    
    stdn,glac,gsmb,asmb,adyn = get_vars(scen)

    stdnglac,stdnglac3 = calc_corr(stdn,glac)
    stdngsmb,stdngsmb3 = calc_corr(stdn,gsmb)
    stdnasmb,stdnasmb3 = calc_corr(stdn,asmb)
    stdnadyn,stdnadyn3 = calc_corr(stdn,adyn)
    glacgsmb,glacgsmb3 = calc_corr(glac,gsmb)
    glacasmb,glacasmb3 = calc_corr(glac,asmb)
    glacadyn,glacadyn3 = calc_corr(glac,adyn)
    gsmbasmb,gsmbasmb3 = calc_corr(gsmb,asmb)
    gsmbadyn,gsmbadyn3 = calc_corr(gsmb,adyn)
    asmbadyn,asmbadyn3 = calc_corr(asmb,adyn)

    #Save
    stdnglac2 = xr.DataArray(stdnglac,dims=('lat','lon'),coords={'lat':lat,'lon':lon})
    stdngsmb2 = xr.DataArray(stdngsmb,dims=('lat','lon'),coords={'lat':lat,'lon':lon})
    stdnasmb2 = xr.DataArray(stdnasmb,dims=('lat','lon'),coords={'lat':lat,'lon':lon})
    stdnadyn2 = xr.DataArray(stdnadyn,dims=('lat','lon'),coords={'lat':lat,'lon':lon})
    glacgsmb2 = xr.DataArray(glacgsmb,dims=('lat','lon'),coords={'lat':lat,'lon':lon})
    glacasmb2 = xr.DataArray(glacasmb,dims=('lat','lon'),coords={'lat':lat,'lon':lon})
    glacadyn2 = xr.DataArray(glacadyn,dims=('lat','lon'),coords={'lat':lat,'lon':lon})
    gsmbasmb2 = xr.DataArray(gsmbasmb,dims=('lat','lon'),coords={'lat':lat,'lon':lon})    
    gsmbadyn2 = xr.DataArray(gsmbadyn,dims=('lat','lon'),coords={'lat':lat,'lon':lon})
    asmbadyn2 = xr.DataArray(asmbadyn,dims=('lat','lon'),coords={'lat':lat,'lon':lon})
    
    ds = xr.Dataset({'stdn_glac':stdnglac2,'stdn_gsmb':stdngsmb2,'stdn_asmb':stdnasmb2,'stdn_adyn':stdnadyn2,'glac_gsmb':glacgsmb2,'glac_asmb':glacasmb2,'glac_adyn':glacadyn2,'gsmb_asmb':gsmbasmb2,'gsmb_adyn':gsmbadyn2,'asmb_adyn':asmbadyn2})
    ds.to_netcdf(f'../data/climcorr_{scen}.nc')
    ds.close()
    
    stdnglac2 = xr.DataArray(stdnglac3,dims=('lat','lon'),coords={'lat':lat,'lon':lon})
    stdngsmb2 = xr.DataArray(stdngsmb3,dims=('lat','lon'),coords={'lat':lat,'lon':lon})
    stdnasmb2 = xr.DataArray(stdnasmb3,dims=('lat','lon'),coords={'lat':lat,'lon':lon})
    stdnadyn2 = xr.DataArray(stdnadyn3,dims=('lat','lon'),coords={'lat':lat,'lon':lon})
    glacgsmb2 = xr.DataArray(glacgsmb3,dims=('lat','lon'),coords={'lat':lat,'lon':lon})
    glacasmb2 = xr.DataArray(glacasmb3,dims=('lat','lon'),coords={'lat':lat,'lon':lon})
    glacadyn2 = xr.DataArray(glacadyn3,dims=('lat','lon'),coords={'lat':lat,'lon':lon})
    gsmbasmb2 = xr.DataArray(gsmbasmb3,dims=('lat','lon'),coords={'lat':lat,'lon':lon})    
    gsmbadyn2 = xr.DataArray(gsmbadyn3,dims=('lat','lon'),coords={'lat':lat,'lon':lon})
    asmbadyn2 = xr.DataArray(asmbadyn3,dims=('lat','lon'),coords={'lat':lat,'lon':lon})
    
    ds = xr.Dataset({'stdn_glac':stdnglac2,'stdn_gsmb':stdngsmb2,'stdn_asmb':stdnasmb2,'stdn_adyn':stdnadyn2,'glac_gsmb':glacgsmb2,'glac_asmb':glacasmb2,'glac_adyn':glacadyn2,'gsmb_asmb':gsmbasmb2,'gsmb_adyn':gsmbadyn2,'asmb_adyn':asmbadyn2})
    ds.to_netcdf(f'../data/climrcov_{scen}.nc')
    ds.close()
    print('Saved',scen)

/Users/erwin/miniconda2/envs/codep/lib/python3.6/site-packages/ipykernel_launcher.py:11: RuntimeWarning: Mean of empty slice
  # This is added back by InteractiveShellApp.init_path()
/Users/erwin/miniconda2/envs/codep/lib/python3.6/site-packages/ipykernel_launcher.py:12: RuntimeWarning: Mean of empty slice
  if sys.path[0] == '':
/Users/erwin/miniconda2/envs/codep/lib/python3.6/site-packages/ipykernel_launcher.py:13: RuntimeWarning: Mean of empty slice
  del sys.path[0]
/Users/erwin/miniconda2/envs/codep/lib/python3.6/site-packages/ipykernel_launcher.py:14: RuntimeWarning: Mean of empty slice
  
/Users/erwin/miniconda2/envs/codep/lib/python3.6/site-packages/ipykernel_launcher.py:15: RuntimeWarning: Mean of empty slice
  from ipykernel import kernelapp as app
/Users/erwin/miniconda2/envs/codep/lib/python3.6/site-packages/numpy/lib/nanfunctions.py:1667: RuntimeWarning: Degrees of freedom <= 0 for slice.
  keepdims=keepdims)
/Users/erwin/miniconda2/envs/codep/lib/python3.6/site-packages/i

got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
Saved rcp45
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
Saved rcp85
